In [1]:
import os
import NNFunctions as NNF
import numpy as np
import pandas as pd
import sklearn.metrics as sklm
from sklearn.model_selection import KFold


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
import torchvision.datasets as dsets
import torchvision.transforms as transforms

In [2]:
data = pd.read_csv("drugConsumption_5Response.csv")
data

,id,Age,Gender,Education,Country,Ethnicity,Neuroticism,Extraversion,Openness,Agreeableness,Conscientiousness,Impulsive,Sensation Seeking,Benzos,Caffeine,Cocaine,LSD,Heroin
0,1.0,0.49788,0.48246,-0.05921,0.96082,0.12600,0.31287,-0.57545,-0.58331,-0.91699,-0.00665,-0.21712,-1.18084,0.0,1.0,0.0,0.0,0.0
1,2.0,-0.07854,-0.48246,1.98437,0.96082,-0.31685,-0.67825,1.93886,1.43533,0.76096,-0.14277,-0.71126,-0.21575,0.0,1.0,1.0,0.0,0.0
2,3.0,0.49788,-0.48246,-0.05921,0.96082,-0.31685,-0.46725,0.80523,-0.84732,-1.62090,-1.01450,-1.37983,0.40148,0.0,1.0,0.0,0.0,0.0
3,4.0,-0.95197,0.48246,1.16365,0.96082,-0.31685,-0.14882,-0.80615,-0.01928,0.59042,0.58489,-1.37983,-1.18084,1.0,1.0,0.0,0.0,0.0
4,5.0,0.49788,0.48246,1.98437,0.96082,-0.31685,0.73545,-1.63340,-0.45174,-0.30172,1.30612,-0.21712,-0.21575,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1880,1884.0,-0.95197,0.48246,-0.61113,-0.57009,-0.31685,-1.19430,1.74091,1.88511,0.76096,-1.13788,0.88113,1.92173,0.0,1.0,0.0,1.0,0.0
1881,1885.0,-0.95197,-0.48246,-0.61113,-0.57009,-0.31685,-0.24649,1.74091,0.58331,0.76096,-1.51840,0.88113,0.76540,0.0,1.0,0.0,1.0,0.0
1882,1886.0,-0.07854,0.48246,0.45468,-0.57009,-0.31685,1.13281,-1.37639,-1.27553,-1.77200,-1.38502,0.52975,-0.52593,1.0,1.0,1.0,0.0,0.0
1883,1887.0,-0.95197,0.48246,-0.61113,-0.57009,-0.31685,0.91093,-1.92173,0.29338,-1.62090,-2.57309,1.29221,1.22470,0.0,1.0,0.0,1.0,0.0


In [3]:
drugs = ["Benzos", "Caffeine","Cocaine","LSD","Heroin"]
freq = [sum(data[i]) for i in drugs]
freq

[535.0, 1824.0, 417.0, 380.0, 118.0]

In [4]:
data = pd.DataFrame.to_numpy(data)

In [5]:
np.random.seed(1)
traindata, testdata = NNF.train_testSplit(data)
print(traindata.shape)
print(testdata.shape)

(1414, 18)
(471, 18)


In [6]:
freq = [sum(traindata[:,i]) for i in range(13,18)]
freq

[400.0, 1367.0, 306.0, 279.0, 85.0]

In [7]:
hiddenlSizes = np.arange(4,22,4)
lrs = np.array([0.3,.01,.001])
print(hiddenlSizes)

[ 4  8 12 16 20]


In [8]:
cverrors = np.zeros((5,4,3))

In [9]:
for i in range(len(hiddenlSizes)):
    hs = hiddenlSizes[i]
    for j in range(len(lrs)):
        print(i,j)
        lr = lrs[j]
        if lr <= 0.001:
            errors = NNF.get_cvError(traindata,lr,hs, epochs=400)
        else:
            errors = NNF.get_cvError(traindata,lr,hs)
        cverrors[i][j] = np.array(list(errors))
        


0 0
0 1
0 2
1 0
1 1
1 2
2 0
2 1
2 2
3 0
3 1
3 2
4 0
4 1
4 2


In [10]:
cverrors

array([[[0.14836629, 0.52617848, 0.6705335 ],
        [0.15445104, 0.52123149, 0.54000643],
        [0.15840563, 0.51982056, 0.52207673],
        [0.        , 0.        , 0.        ]],

       [[0.15147483, 0.52265995, 0.66364016],
        [0.1513395 , 0.52052477, 0.54448178],
        [0.15854698, 0.52123399, 0.50864927],
        [0.        , 0.        , 0.        ]],

       [[0.15571762, 0.51062827, 0.64555981],
        [0.1530321 , 0.52618851, 0.60611879],
        [0.15840563, 0.52335413, 0.5415976 ],
        [0.        , 0.        , 0.        ]],

       [[0.15430619, 0.52124151, 0.65221735],
        [0.15020575, 0.52618349, 0.56020405],
        [0.15911235, 0.52123399, 0.51600807],
        [0.        , 0.        , 0.        ]],

       [[0.15841415, 0.50850813, 0.62770784],
        [0.15176102, 0.52264742, 0.55509477],
        [0.15812245, 0.52406586, 0.50220062],
        [0.        , 0.        , 0.        ]]])

In [11]:
hiddenlSizes = np.arange(4,12,2)
lrs = np.array([0.4,0.3])
print(hiddenlSizes)

[ 4  6  8 10]


In [12]:
cverrors1 = np.zeros((4,2,3))

In [13]:
for i in range(len(hiddenlSizes)):
    hs = hiddenlSizes[i]
    for j in range(len(lrs)):
        print(i,j)
        lr = lrs[j]
        if lr <= 0.001:
            errors = NNF.get_cvError(traindata,lr,hs, epochs=400)
        else:
            errors = NNF.get_cvError(traindata,lr,hs)
        cverrors1[i][j] = np.array(list(errors))

0 0
0 1
1 0
1 1
2 0
2 1
3 0
3 1


In [14]:
cverrors1

array([[[0.15162018, 0.52619352, 0.68259849],
        [0.14850713, 0.53042879, 0.67762332]],

       [[0.14779691, 0.5304338 , 0.67067687],
        [0.15091046, 0.5219382 , 0.66405005]],

       [[0.1493597 , 0.53041626, 0.66613068],
        [0.15190186, 0.52406335, 0.66312134]],

       [[0.15543643, 0.51982558, 0.66839908],
        [0.15147583, 0.52760945, 0.66520694]]])

In [ ]:
## visualize cross validation results

In [ ]:
## test model we chose: 
# hidden layer is 4, lr = 0.3